# Trace Estimator  
https://doi.org/10.1016/0377-0427(96)00018-0 
## b) Algorithm 1 with Gauss-Radau quadrature

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from warnings import warn
#for part e)
from matrices import heat_flow_function, Pei_function, VFH_function, Poisson_function, wathen_ge, Lehmer_function 
import time 
from scipy.linalg import block_diag
from helpers import *

In [14]:
def algorithm_1(A, u, function, maxit=50, epsilon=1e-5):
    '''
    Implements algorithm 1 from Bai et al. It computes a lower/upper bound of the quantity u^T f(A) u by using the Gauss-Radau rules
    INPUT:
    - A: a symmetric positive definite matrix of size n times n for some n, with eigenvalues in [a,b]
    - u: vector of size n 
    - f: smooth function in the interval [a,b]
    - maxit: maximum number of iteration
    - epsilon: tolerance between two iterations
    
    OUTPUT:
    - [U,L]: Upper and Lower bound of the quantity u^T f(A) u by using the Gauss-Radau rule.
    '''
    # Remark 1:
    interval = Gerschgorin(A)

    if interval[0]<=0:
        interval[0] = 1e-4
    print("a=", interval[0])
    print("b=", interval[1])
    if (np.linalg.eigvals(A)<=0).any():
        warn("The matrix A should be positive definite.")
        print("eigenvalues of A:", np.linalg.eigvals(A))
    if not (A==A.T).all():
        warn("A is not symmetric. Please choose A such that A=A.T")
        print("A =",A)

    # set the first variables
    x_j1 = u/np.linalg.norm(u)    
    gamma_j1 = 0.0
    I_j = np.zeros(2)
    I_j1 = np.zeros(2)

    # Save X as a matrix (for re-orthogonalization)
    n=len(u) 
    X=np.zeros((n, maxit+1))
    X[:,0] = x_j1

    for j in range(maxit):
        w = A@X[:,j]
        alpha_j=X[:,j].T@w
        r_j = w - alpha_j*X[:,j]
        if j>0:
            r_j = r_j - gamma_j1*X[:,j-1]
        
        #reorthogonlization to avoid roundoff error encured by Lanczos
        alphas=X.T@r_j
        r_j=r_j-X@alphas
        alpha_j=alpha_j+alphas[-1]
        gamma_j = np.linalg.norm(r_j)

        # build T_j:
        if j==0:
            T_j = np.array([alpha_j])
        else:
            # horizontal array [0, ..., 0, gamma_{j-1}]
            temp_h = np.expand_dims(np.zeros(T_j.shape[0]),1)
            temp_h[-1] = gamma_j1
            # vertical array [0, ..., 0, gamma_{j-1}, alpha_j].T
            temp_v = np.expand_dims(np.zeros(T_j.shape[0] + 1),1)
            temp_v[-1] = alpha_j
            temp_v[-2] = gamma_j1
            # new T_j:
            T_j = np.hstack((np.vstack((T_j, temp_h.T)), temp_v))
        
        # for Gauss Radau, a or b have to be zeros of the polynomial, i.e. must be eigenvalues of T_tilda_{j'}:
        for i in range(2): # for lower and upper bounds
            T_tilda_j = T_tilda(T_j, gamma_j, interval[i])
            

            # compute eigenvalues of T_tilda_j:            
            theta_k, eig_vec = np.linalg.eigh(T_tilda_j)
            w_k_square = eig_vec[0, :]*eig_vec[0,:]
            I_j[i] = function(theta_k).dot(w_k_square)
                
        print("at j=", j,", I_j=", I_j)
        
        if (j>0) & (np.linalg.norm(I_j - I_j1) <= epsilon*np.linalg.norm(I_j)):
            break
        
        x_j1 = r_j/gamma_j
        X[:,j+1]=x_j1.copy()
        I_j1 = I_j.copy()
        gamma_j1 = gamma_j.copy()
        
        if not check_ortho(X[:,0:j+1]):
            warn("The algorithm does not build an orthonormal basis, at j ="+str(j))
    return u.dot(u)*I_j   

In [15]:
#the Poisson matrix
k=12
A= Poisson_function(k=k)
n=k**2
print("n=",n)
u = np.zeros(n)
u[4] = 1
u = np.random.randn(n)
def f(x):
    return 1/x
tol = 1e-5
L = algorithm_1(A=A, u=u, function=f, maxit=20, epsilon=tol)

print("bounds [U,L]=", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

n= 144
a= 0.0001
b= 8.0
at j= 0 , I_j= [1.69622947e+03 2.61663844e-01]
at j= 1 , I_j= [5.55469422e+02 3.05670856e-01]
at j= 2 , I_j= [246.28161313   0.32952317]
at j= 3 , I_j= [117.93950865   0.34189629]
at j= 4 , I_j= [58.54176922  0.34809938]
at j= 5 , I_j= [32.49310945  0.35170053]
at j= 6 , I_j= [19.01129181  0.35398122]
at j= 7 , I_j= [10.78564133  0.35527944]
at j= 8 , I_j= [6.413047   0.35605811]
at j= 9 , I_j= [3.7471052 0.3564779]
at j= 10 , I_j= [2.22785702 0.35669129]
at j= 11 , I_j= [1.61114896 0.35688984]
at j= 12 , I_j= [1.22604849 0.35704215]
at j= 13 , I_j= [0.95096225 0.35714449]
at j= 14 , I_j= [0.80386572 0.35723666]
at j= 15 , I_j= [0.6905155  0.35730499]
at j= 16 , I_j= [0.58032184 0.3573429 ]
at j= 17 , I_j= [0.48587283 0.3573606 ]
at j= 18 , I_j= [0.43090949 0.35737067]
at j= 19 , I_j= [0.39606578 0.35737554]
bounds [U,L]= [60.99403514 55.03574805]
exact value: 55.036147646378254
L<I_ex: True
U>I_ex: True
L<U = True


In [16]:
#the Pei matrix
n=120
A=Pei_function(alpha=5, n=n)
def f(x):
    return 1/x
u = np.zeros(n)
u[1] = 1
#u = np.random.randn(n)
tol = 1e-3
L = algorithm_1(A=A, u=u, function=f, maxit=200, epsilon=tol)

print("bounds [U,L]=", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

a= 0.0001
b= 125.0
at j= 0 , I_j= [7.67748778e+03 1.98400000e-01]
at j= 1 , I_j= [0.1984 0.1984]
at j= 2 , I_j= [0.1984 0.1984]
bounds [U,L]= [0.1984 0.1984]
exact value: 0.19840000000000016
L<I_ex: True
U>I_ex: False
L<U = True


In [17]:
n = 5
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

vec1 = -1*np.ones(n-1)
vec2 = 2*np.ones(n)
vec3 = -1*np.ones(n-1)
A = tridiag(vec1, vec2, vec3)

u = np.zeros(n)
u[1] = 1
# u = np.random.randn(n)
tol = 1e-2
L = algorithm_1(A=A, u=u, function=f, maxit=200, epsilon=tol)

print("bounds [L, U]=", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

a= 0.0001
b= 4.0
at j= 0 , I_j= [3.33377778e+03 7.50000000e-01]
at j= 1 , I_j= [1.42946938e+03 1.08333333e+00]
at j= 2 , I_j= [770.31951694   1.25      ]
at j= 3 , I_j= [1.33333333 1.33333333]
at j= 4 , I_j= [1.33333333 1.33333333]
bounds [L, U]= [1.33333333 1.33333333]
exact value: 1.333333333333333
L<I_ex: False
U>I_ex: True
L<U = False


### e) Numerical experiments (first draft)

In [6]:
import time 
import numpy as np
from scipy.linalg import block_diag

In [7]:
#cell to be rewritten, temporary assignement to A
A=np.eye(3,3) 

**The different matrices they experimented on in the paper:**

In [8]:
#example 1: the heat flow matrix

In [2]:
n=2
nu=3 #some random values for now, nu>0 implies heat_flow_matrix is positive definite which is something we want
heat_flow_matrix=heat_flow_function(nu,n)
heat_flow_matrix

array([[13., -3., -3.,  0.],
       [-3., 13.,  0., -3.],
       [-3.,  0., 13., -3.],
       [ 0., -3., -3., 13.]])

In [17]:
#the Pei matrix
alpha=2
n=4
Pei_matrix=Pei_function(alpha, n)

u = np.random.rand(n)
U, L = algorithm_1(A=Pei_matrix, u=u, function=f, maxit=200, epsilon=tol)
print("L=", L)
print("U=", U)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(Pei_matrix)@u)
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("is the exact value between L and U:", I_ex<U and L<I_ex)

a = 0.0001
b = 6.0
Tolerance 1e-13 was reached at iteration 2
L= 0.16153600220772452
U= 0.16153600220772452
exact value: 0.19887453815178047
is the exact value between L and U: False


In [ ]:
#running time of algo 2

start=time.time()
E,L,U=algorithm_2(A,m,p) #for some m,p to tune 
execution_algo_2=time.time()-start 

In [4]:
#running time using built in numpy functions
A=heat_flow_matrix
start=time.time()
Tr_A_inv=np.trace(np.linalg.inv(A))
execution_built_in=time.time()-start 
print('execution time', execution_built_in)

execution time 0.0012359619140625


In [6]:
#solving n linear equations

#(Armelle: in my opinion, when they say compute trace(inv(A)) using n linear equations, they mean computing 
#e_i.T inv(A) e_i for i in {1,...,n} )

start=time.time()
n=A.shape[0]

trace=0
for i in range(n):
    e=np.zeros(n)
    e[i]=1
    trace+=e.T@np.linalg.inv(A)@e
execution_linear_equations=time.time()-start 

print('execution time', execution_linear_equations)

execution time 0.03495073318481445


In [ ]:
#using algorithm 1

n=A.shape[0]
start=time.time()
trace=0
for i in range (n):
    trace+= #output of algo 1 (I guess here also parameter values to tune)
execution_algo_1=time.time()-start 